In [5]:
"""
Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
체인을 수동으로 구현해야 합니다.
체인에 ConversationBufferMemory를 부여합니다.
이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
체인에 다음 질문을 합니다:
    Aaronson 은 유죄인가요?
    그가 테이블에 어떤 메시지를 썼나요?
    Julia 는 누구인가요?
"""

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("../files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context.
            If you don't know the answer just say you don't know, don't make it up:
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)


def load_memory(_):
    x = memory.load_memory_variables({})
    return {"history": x["history"]}

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(chat_history=load_memory),
    }
    | prompt
    | llm
)

a1 = chain.invoke("Aaronson 은 유죄인가요?")
print(a1)
a2 = chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")
print(a2)
a3 = chain.invoke("Julia 는 누구인가요?")
print(a3)


NameError: name 'ConversationBufferMemory' is not defined